In [1]:
import os
import glob
import datetime
import warnings

warnings.simplefilter('ignore')

# from multiprocessing import Pool
import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

# Other packages
import netCDF4
import pytmatrix
import numpy as np
import xarray as xr

from netCDF4 import num2date, date2num
from scipy import interpolate
from pytmatrix import orientation, radar, tmatrix_aux, refractive
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix.tmatrix import TMatrix, Scatterer
from pytmatrix.tmatrix_psd import TMatrixPSD, GammaPSD

In [2]:
def read_ascii_file(filename):
    """
    Read ascii data disdrometer file.

    Parameters:
    ===========
        filename: str
            Input file name
            
    Returns:
    ========
    diams: ndarray
        Diameters.
    disdro_data: dict
        All others disdro parameters.
    PSD_raw_count: ndarray
        Disdro psd.
    """
    # Opening File
    with open(filename, 'r') as fid:
        content = fid.read()
        lines = content.splitlines()

    # First line = diameters.
    line = lines[0].split()
    diams = np.array(line, dtype=float)
    
    # Total array length
    len_line = len(lines) - 1

    # Initializing empty array
    line_count = [None]*(len_line)
    date = [None]*(len_line)
    latitude = [None]*(len_line)
    longitude = [None]*(len_line)
    disdrometer_rain_probability = [None]*(len_line)
    disdrometer_snow_probability = [None]*(len_line)
    disdrometer_mixedphase_probability = [None]*(len_line)
    disdrometer_precip_flag = [None]*(len_line)
    disdrometer_precip_flag2 = [None]*(len_line)
    disdrometer_nb_bins = [None]*(len_line)
    disdrometer_nb_particles = [None]*(len_line)
    disdrometer_precip_rate = [None]*(len_line)
    disdrometer_reflectivity = [None]*(len_line)
    disdrometer_dBR = [None]*(len_line)
    disdrometer_dBZ = [None]*(len_line)
    disdrometer_relative_wind_speed = [None]*(len_line)
    disdrometer_reference_voltage = [None]*(len_line)
    PSD_raw_count = np.zeros((len_line, 128))

    # Parsing line by line
    for cnt, myline in enumerate(lines[1:]):
        # Splitting empty space.
        line = myline.split()

        line_count[cnt] = int(line[0])
        date[cnt] = datetime.datetime.strptime(line[1] + line[2], "%d%m%Y%H%M")
        latitude[cnt] = float(line[6])
        longitude[cnt] = float(line[7])
        disdrometer_rain_probability[cnt] = float(line[8])
        disdrometer_snow_probability[cnt] = float(line[9])
        disdrometer_mixedphase_probability[cnt] = float(line[10])
        disdrometer_precip_flag[cnt] = int(line[11])
        disdrometer_precip_flag2[cnt] = int(line[12])
        disdrometer_nb_bins[cnt] = int(line[13])
        disdrometer_nb_particles[cnt] = int(line[14])
        disdrometer_precip_rate[cnt] = float(line[15])
        disdrometer_reflectivity[cnt] = float(line[16])
        disdrometer_dBR[cnt] = float(line[17])
        disdrometer_dBZ[cnt] = float(line[18])
        disdrometer_relative_wind_speed[cnt] = float(line[19])
        disdrometer_reference_voltage[cnt] = float(line[20])

        PSD_raw_count[cnt, :] = np.array(line[21:], dtype=float)

    disdro_data = dict()
    disdro_data["line_count"] = line_count
    disdro_data["date"] = date
    disdro_data["latitude"] = latitude
    disdro_data["longitude"] = longitude
    disdro_data["disdrometer_rain_probability"] = disdrometer_rain_probability
    disdro_data["disdrometer_snow_probability"] = disdrometer_snow_probability
    disdro_data["disdrometer_mixedphase_probability"] = disdrometer_mixedphase_probability
    disdro_data["disdrometer_precip_flag"] = disdrometer_precip_flag
    disdro_data["disdrometer_precip_flag2"] = disdrometer_precip_flag2
    disdro_data["disdrometer_nb_bins"] = disdrometer_nb_bins
    disdro_data["disdrometer_nb_particles"] = disdrometer_nb_particles
    disdro_data["disdrometer_precip_rate"] = disdrometer_precip_rate
    disdro_data["disdrometer_reflectivity"] = disdrometer_reflectivity
    disdro_data["disdrometer_dBR"] = disdrometer_dBR
    disdro_data["disdrometer_dBZ"] = disdrometer_dBZ
    disdro_data["disdrometer_relative_wind_speed"] = disdrometer_relative_wind_speed
    disdro_data["disdrometer_reference_voltage"] = disdrometer_reference_voltage

    return diams[21:], disdro_data, PSD_raw_count

In [22]:
def drop_axis_ratio(D_eq):
    """
    Axis ratio of drops with respect to their diameter.

    Parameter:
    ==========
        D_eq: float
            Drop diameter.
    Return:
    =======
        axratio: float
            Axis ratio of drop.
    """
    if D_eq < 0.7:
        axratio = 1.0  # Spherical
    elif D_eq < 1.5:
        axratio = 1.173 - 0.5165 * D_eq + 0.4698 * D_eq**2 - 0.1317 * D_eq**3 - 8.5e-3 * D_eq**4
    else:
        axratio = 1.065 - 6.25e-2 * D_eq - 3.99e-3 * D_eq**2 + 7.66e-4 * D_eq**3 - 4.095e-5 * D_eq**4

    return 1.0 / axratio


def buffer(d_diameters, d_densities):
    
    if len(d_diameters) != len(d_densities):
        print(len(d_diameters), len(d_densities))
        raise IndexError("Not the same dim")
        
    try:
        dbz, zdr, kdp, atten_spec, atten_spec_v = scatter_off_2dvd_packed(d_diameters, d_densities)
    except Exception:
        raise
    
    return dbz, zdr, kdp, atten_spec, atten_spec_v


def radar_band_name(wavelength):
    """

    Parameters:
    ===========
        wavelength: float
            Radar wavelength in mm.

    Returns:
    ========
        freqband: str
            Frequency band name.
    """

    if wavelength >= 100:
        return "S"
    elif wavelength >= 40:
        return "C"
    elif wavelength >= 30:
        return "X"
    elif wavelength >= 20:
        return "Ku"
    elif wavelength >= 7:
        return "Ka"
    else:
        return "W"

    return None


def scatter_off_2dvd_packed(d_diameters, d_densities):
    """
    Computing the scattering properties of homogeneous nonspherical scatterers with the T-Matrix method.

    Parameters:
    ===========
        d_diameters: array
            Drop diameters in mm! (or else returns values won't be with proper units.)
        d_densities: array
            Drop densities.

    Returns:
    ========
        dbz: array
            Horizontal reflectivity.
        zdr: array
            Differential reflectivity.
        kdp: array
            Specific differential phase (deg/km).
        atten_spec: array
            Specific attenuation (dB/km).
    """
    # Function interpolation.
    mypds = interpolate.interp1d(d_diameters, d_densities, bounds_error=False, fill_value=0.0)
    SCATTERER.psd = mypds  # GammaPSD(D0=2.0, Nw=1e3, mu=4)

    # Obtaining reflectivity and ZDR.
    dbz = 10 * np.log10(radar.refl(SCATTERER))  # in dBZ
    zdr = 10 * np.log10(radar.Zdr(SCATTERER))  # in dB

    # Specific attenuation and KDP.
    SCATTERER.set_geometry(tmatrix_aux.geom_horiz_forw)
    atten_spec = radar.Ai(SCATTERER)  # in dB/km
    atten_spec_v = radar.Ai(SCATTERER, h_pol=False)  # in dB/km
    kdp = radar.Kdp(SCATTERER)  # in deg/km

    return dbz, zdr, kdp, atten_spec, atten_spec_v


In [23]:
def write_netcdf(outfilename, time, diameter, PSD_raw_count, dbz, zdr, kdp, atten_spec, atten_spec_v):
    '''
    Write output netCDF dataset.

    Parameters:
    ===========
    outfilename: str
    time: ndarray
        time
    diameter: ndarray
        diameter
    PSD_raw_count: ndarray
        Concentration number
    dbz: ndarray
        Reflectivity
    zdr: ndarray
        Differential reflectivity
    kdp: ndarray
        Specific differential phase
    atten_spec: ndarray
        Specific attenuation
    atten_spec_v: ndarray
        Vertical specific attenuation 
    '''
    dset = xr.Dataset({'time': (('time'), time),
                        'diameter': (('diameter'), diameter),
                        'concentration_number': (("time", "diameter"), PSD_raw_count),
                        'DBZ': (('time'), dbz),
                        'ZDR': (('time'), zdr),
                        'KDP': (('time'), kdp),
                        'ATTEN_SPEC': (('time'), atten_spec),
                        'ATTEN_SPEC_V': (('time'), atten_spec_v)})

    dset.diameter.attrs['units'] = "mm"
    dset.DBZ.attrs['units'] = "dBZ"
    dset.ZDR.attrs['units'] = "dB"
    dset.KDP.attrs['units'] = "deg/km"
    dset.ATTEN_SPEC.attrs['units'] = "dB/km"
    dset.ATTEN_SPEC_V.attrs['units'] = "dB/km"

    dset.DBZ.attrs["description"] = "Horizontal reflectivity"
    dset.ZDR.attrs["description"] = "Differential reflectivity"
    dset.KDP.attrs["description"] = "Specific differential phase "
    dset.ATTEN_SPEC.attrs["description"] = "Specific attenuation for the horizontal reflectivity"
    dset.ATTEN_SPEC_V.attrs["description"] = "Specific attenuation for the vertical reflectivity"

    dset.to_netcdf(outfilename)

    return None

In [24]:
def main(input_file, freq_band):
    # Generating output file name:
    letter_band = radar_band_name(freq_band)
    outfile = os.path.basename(input_file)    
    outfile = outfile.replace("_psd", f"_PSD_TMATRIX_{letter_band}-band").replace(".txt", ".nc")
    outfilename = os.path.join(OUTDIR, outfile)    
    if os.path.exists(outfilename):
        print("Output file already exists. Doing nothing.")
        return None

    # Read data.
#     diameter_bin_size, disdro_data, PSD_raw_count = read_ascii_file(input_file)
    print("input file {} read.".format(input_file))    

    # Build argument list for multiprocessing.
    myargs = [(diameter_bin_size, PSD_raw_count[cnt, :]) for cnt in range(0, len(PSD_raw_count))]
    bag = db.from_sequence(myargs).starmap(buffer)
    with ProgressBar():
        rslt = bag.compute()

    # Unpack and save results
    dbz, zdr, kdp, atten_spec, atten_spec_v = zip(*rslt)
    dbz = np.array(dbz)
    zdr = np.array(zdr)
    kdp = np.array(kdp)
    atten_spec = np.array(atten_spec)
    atten_spec_v = np.array(atten_spec_v)
    print("T-Matrix computation finished.")

    time = np.array(disdro_data['date'], dtype='datetime64')
    print("The output file will be {}.".format(outfilename))
    write_netcdf(outfilename, time, diameter_bin_size, PSD_raw_count, dbz, zdr, kdp, atten_spec, atten_spec_v)
    print("Output file {} written.".format(outfilename))        

    return None

In [25]:
TIME_UNIT = "seconds since 1970-01-01 00:00"
OUTDIR = "."
# if not os.path.isdir(OUTDIR):
#     os.mkdir(OUTDIR)

# Radar band in mm.
for RADAR_BAND in [tmatrix_aux.wl_S, tmatrix_aux.wl_C, tmatrix_aux.wl_X, tmatrix_aux.wl_Ku, tmatrix_aux.wl_Ka, tmatrix_aux.wl_W]: #
    print("Looking at wavelength {} mm.".format(RADAR_BAND))
    # Invoking T-Matrix scatterer.
    SCATTERER = Scatterer(wavelength=RADAR_BAND, m=refractive.m_w_10C[RADAR_BAND])

    # PSDIntegrator classfrom pytmatrix
    SCATTERER.psd_integrator = PSDIntegrator()

    # Defining the axis ratio of drops.
    SCATTERER.psd_integrator.axis_ratio_func = lambda D: drop_axis_ratio(D)
    SCATTERER.psd_integrator.D_max = 8
    SCATTERER.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)
    SCATTERER.or_pdf = orientation.gaussian_pdf(10.0)
    SCATTERER.orient = orientation.orient_averaged_fixed
    SCATTERER.psd_integrator.init_scatter_table(SCATTERER)
    
    flist = glob.glob("/g/data/kl02/vhl548/data_for_others/disdro2/*psd_na.txt")    
    for infile in flist:    
        main(infile, RADAR_BAND)
        break

Looking at wavelength 111.0 mm.
Output file already exists. Doing nothing.
Looking at wavelength 53.5 mm.
input file /g/data/kl02/vhl548/data_for_others/disdro2/joint_investigator_disdro_2017V02-2017V02_psd_na.txt read.
[########################################] | 100% Completed |  0.5s
T-Matrix computation finished.
The output file will be ./joint_investigator_disdro_2017V02-2017V02_PSD_TMATRIX_C-band_na.nc.
Output file ./joint_investigator_disdro_2017V02-2017V02_PSD_TMATRIX_C-band_na.nc written.
Looking at wavelength 33.3 mm.
input file /g/data/kl02/vhl548/data_for_others/disdro2/joint_investigator_disdro_2017V02-2017V02_psd_na.txt read.
[########################################] | 100% Completed |  0.6s
T-Matrix computation finished.
The output file will be ./joint_investigator_disdro_2017V02-2017V02_PSD_TMATRIX_X-band_na.nc.
Output file ./joint_investigator_disdro_2017V02-2017V02_PSD_TMATRIX_X-band_na.nc written.
Looking at wavelength 22.0 mm.
input file /g/data/kl02/vhl548/data_fo